In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

In [4]:
df1 = pd.read_csv("dictionary_verified_top_nsfw_words.csv")
df2 = pd.read_csv("dot05_nsfw_words.csv")

In [46]:
data = pd.read_csv("diffusion_db_unaltered.csv")
data = data.dropna()

In [68]:
# Simple vectorization: Looking for the word 'boobs, blood, scary, robot'
vectors = {
    'vector': []
}

for prompt in data['prompt']:
    if "boobs, blood, scary, robot" in prompt:
        vectors['vector'].append(1)
    else:
        vectors['vector'].append(0)

data['vector'] = vectors['vector']

In [69]:
data.head()

,prompt,image_nsfw,prompt_nsfw,vector
0,"a portrait of a female robot made from code, v...",0.554853,0.001621,0
1,a portrait of a female robot made from a cloud...,0.153645,0.000707,0
2,"only memories remain, trending on artstation",0.062496,0.000425,0
3,dream swimming pool with nobody,0.030799,0.003586,0
4,a dog doing weights. epic oil painting.,0.181035,0.030822,0


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91564 entries, 0 to 91188
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    91564 non-null  object
 1   count   91564 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.1+ MB


In [5]:
df = pd.concat([df1, df2]).drop(columns=["Unnamed: 0"]).dropna()

In [65]:
df.head()

,word,count
0,knave,2
1,spake,6
2,wordsmith,4
3,tween,2
4,gou,3


In [39]:
vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(df['word'])

In [41]:
X_text

<91564x75400 sparse matrix of type '<class 'numpy.float64'>'
	with 91542 stored elements in Compressed Sparse Row format>

In [49]:
scaler = StandardScaler()
X_count = scaler.fit_transform(df[['count']])

In [50]:
X_features = [(X_text[i], X_count[i]) for i in range(len(df))]

In [51]:
y = np.array([0] * (len(df1)) + [1] * (len(df2)))


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91564 entries, 0 to 91188
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    91564 non-null  object
 1   count   91564 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.1+ MB
